<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=830e353d337c1cb94b58148859b96779aa5f0e65377f3d1bf07dd85a5c6f97d7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-05 10:35:23
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: -16.79 K (-0.12%)
Current PnL: -24.89 L (-15.87%)
CY Booked + Current PnL: -10.60 L (-6.76%)
-------------------
Total profit:  1.89 L
Total loss:  -26.78 L
-------------------
Total Booked + Current PnL: 16.38 L (12.62%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.91%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.86 L (62.32%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.98,14.10,4.90,19.69,10720.0,27033.0,218775.0,2.93,67.0,X-MC,2.97,78.0,2.52,1.52,21.85,XY25,NTT,AC
51,NESTLEIND,1377.00,1.61,14.63,5.90,21.40,17952.0,38844.0,304270.0,6.33,66.0,X-LC,5.00,12.0,2.16,2.11,21.89,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,1.92,-18.63,23.00,0.09,18861.0,-18771.0,82006.0,90.39,48.0,M-SC,3.48,253.0,-1.00,0.57,7.74,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.27,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,82.0,L-LC,5.01,179.0,0.35,1.10,27.27,XY25,ATH,MINING
49,MASFIN,398.61,-0.85,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,55.0,H-SC,6.20,164.0,-0.05,0.67,40.65,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,4.81,-34.13,111.94,39.61,75582.0,-34980.0,67520.0,-55.45,51.0,H-SC,32.89,135.0,-0.46,0.47,9.29,AR,ATH,MISC
10,BANDHANBNK,400.00,2.58,-28.71,169.92,92.42,337419.0,-79984.0,198575.0,39.37,51.0,H-SC,9.50,170.0,-0.24,1.38,14.76,XY24,NTT,BANKS
34,HONAUT,58357.33,2.13,-15.66,69.94,43.32,96070.0,-25512.0,137360.0,-27.57,55.0,X-SC,7.75,90.0,-0.27,0.95,5.47,X40N,ATH,ELECTRICAL
82,VOLTAS,1530.00,1.98,14.10,4.90,19.69,10720.0,27033.0,218775.0,2.93,67.0,X-MC,2.97,78.0,2.52,1.52,21.85,XY25,NTT,AC
77,TTKPRESTIG,770.00,1.92,-18.63,23.00,0.09,18861.0,-18771.0,82006.0,90.39,48.0,M-SC,3.48,253.0,-1.00,0.57,7.74,OX40N,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1972.00,-2.60,8.76,23.43,34.24,79735.0,27422.0,340310.0,-15.29,47.0,X-LC,7.38,2.0,0.34,2.36,17.77,X40,NTT,IT
32,HCLTECH,1922.01,-2.26,4.04,19.89,24.73,50061.0,9763.0,251687.0,8.52,41.0,X-LC,6.68,7.0,0.20,1.75,19.89,X40,ATH,IT
54,QUESS,424.00,-2.21,-29.15,101.63,42.84,46805.0,-18952.0,46054.0,-53.72,48.0,X-SC,29.90,83.0,-0.40,0.32,4.19,XY24,NTT,MISC
84,WIPRO,420.00,-2.12,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,53.0,M-LC,5.66,99.0,0.12,1.27,14.66,XR,NTT,IT
55,RAJESHEXPO,518.00,-1.81,-65.67,191.49,0.07,90178.0,-90084.0,47093.0,1626.01,38.0,L-SC,4.32,268.0,-1.00,0.33,17.12,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.36,-1.00,110.01,107.91,160801.0,-1475.0,146169.0,-19.34,56.0,M-SC,10.66,216.0,-0.01,1.01,8.99,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.80,-18.30,22.42,0.02,31974.0,-31947.0,142612.0,117.63,43.0,M-SC,6.94,234.0,-1.00,0.99,24.05,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.92,-18.63,23.00,0.09,18861.0,-18771.0,82006.0,90.39,48.0,M-SC,3.48,253.0,-1.00,0.57,7.74,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.65,-20.06,41.95,13.47,90431.0,-54099.0,215568.0,-67.15,58.0,H-SC,5.67,158.0,-0.60,1.50,9.61,XY24,NTT,PAINTS
66,SIS,528.00,-0.12,-22.17,55.80,21.26,48222.0,-24612.0,86420.0,2041.45,58.0,H-SC,4.45,166.0,-0.51,0.60,16.76,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.03,3.07,17.14,20.74,35893.0,6242.0,209412.0,35.83,73.0,M-MC,7.38,189.0,0.17,1.45,14.01,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.27,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,82.0,L-LC,5.01,179.0,0.35,1.10,27.27,XY25,ATH,MINING
84,WIPRO,420.00,-2.12,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,53.0,M-LC,5.66,99.0,0.12,1.27,14.66,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-2.12,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,53.0,M-LC,5.66,99.0,0.12,1.27,14.66,XR,NTT,IT
25,FINCABLES,1641.55,-0.36,-1.00,110.01,107.91,160801.0,-1475.0,146169.0,-19.34,56.0,M-SC,10.66,216.0,-0.01,1.01,8.99,OX40N,ATH,CABLES
49,MASFIN,398.61,-0.85,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,55.0,H-SC,6.20,164.0,-0.05,0.67,40.65,XR,ATH,FINANCE
38,INDIAMART,4810.62,0.90,-5.58,118.93,106.72,138502.0,-6879.0,116457.0,-53.30,41.0,H-SC,6.65,139.0,-0.05,0.81,16.50,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.17,-3.15,40.04,35.63,81272.0,-6597.0,202977.0,-16.74,47.0,H-MC,2.93,119.0,-0.08,1.41,13.00,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.93,-13.41,27.94,10.78,58141.0,-32224.0,208094.0,-48.51,19.0,X-LC,3.73,5.0,-0.55,1.44,0.93,X40,NTT,FMCG
23,DMART,5391.45,-1.71,-12.11,47.46,29.60,104114.0,-30228.0,219372.0,-25.02,28.0,X-LC,9.41,19.0,-0.29,1.52,7.41,X40N,ATH,FMCG
22,DIXON,18931.72,0.37,-15.34,55.05,31.26,107546.0,-35408.0,195360.0,-60.43,34.0,X-MC,17.47,56.0,-0.33,1.36,3.76,X40N,ATH,IT
7,AWL,485.00,-0.29,-23.64,105.87,57.19,263636.0,-77108.0,249019.0,-63.88,34.0,X-MC,7.70,58.0,-0.29,1.73,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.29,-9.72,44.67,30.62,95603.0,-23034.0,214020.0,-35.86,35.0,X-MC,15.66,55.0,-0.24,1.48,0.30,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,0.51,2.01,18.38,20.77,43902.0,4712.0,238856.0,-6.85,42.0,X-MC,1.04,62.0,0.11,1.66,9.07,X40N,NTT,BREWERIES
65,SIEMENS,4671.50,0.34,-16.49,50.30,25.51,78169.0,-30690.0,155405.0,-4.56,47.0,H-LC,1.29,49.0,-0.39,1.08,14.52,AR,ATH,ELECTRICAL
20,DABUR,735.00,1.01,3.21,39.23,43.70,100441.0,7964.0,256032.0,-5.22,71.0,X-MC,1.90,72.0,0.08,1.78,18.74,XY24,BTT,FMCG
12,BERGEPAINT,680.00,0.10,-6.54,29.62,21.14,62932.0,-14876.0,212463.0,-13.17,39.0,X-MC,2.24,74.0,-0.24,1.47,17.99,XY24,NTT,PAINTS
81,VBL,671.64,-1.54,-1.79,38.14,35.67,118309.0,-5646.0,310196.0,-13.42,56.0,X-LC,2.56,4.0,-0.05,2.15,11.82,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.29,-23.64,105.87,57.19,263636.0,-77108.0,249019.0,-63.88,34.0,X-MC,7.70,58.0,-0.29,1.73,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.29,-9.72,44.67,30.62,95603.0,-23034.0,214020.0,-35.86,35.0,X-MC,15.66,55.0,-0.24,1.48,0.30,X40,ATH,APPARELS
43,ITC,452.00,0.93,-13.41,27.94,10.78,58141.0,-32224.0,208094.0,-48.51,19.0,X-LC,3.73,5.0,-0.55,1.44,0.93,X40,NTT,FMCG
11,BATAINDIA,2096.00,0.54,-38.38,119.41,35.20,94680.0,-49380.0,79290.0,3.97,44.0,X-SC,17.16,92.0,-0.52,0.55,1.84,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.53,-26.12,122.18,64.15,214792.0,-62151.0,175800.0,-56.75,45.0,X-SC,5.98,82.0,-0.29,1.22,1.90,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.21,-29.15,101.63,42.84,46805.0,-18952.0,46054.0,-53.72,48.0,X-SC,29.90,83.0,-0.40,0.32,4.19,XY24,NTT,MISC
58,RELAXO,1176.00,0.50,-47.51,190.16,52.31,144894.0,-68964.0,76196.0,-44.15,47.0,X-SC,7.47,91.0,-0.48,0.53,1.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.54,-38.38,119.41,35.20,94680.0,-49380.0,79290.0,3.97,44.0,X-SC,17.16,92.0,-0.52,0.55,1.84,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.68,2.32,21.27,24.08,27628.0,2942.0,129892.0,-5.97,60.0,X-SC,7.23,89.0,0.11,0.90,22.39,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,2.13,-15.66,69.94,43.32,96070.0,-25512.0,137360.0,-27.57,55.0,X-SC,7.75,90.0,-0.27,0.95,5.47,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-1.06,-11.56,36.49,20.71,122057.0,-43742.0,334495.0,-23.95,49.0,X-LC,2.71,1.0,-0.36,2.32,11.77,X40,ATH,IT
41,INFY,1972.00,-2.60,8.76,23.43,34.24,79735.0,27422.0,340310.0,-15.29,47.0,X-LC,7.38,2.0,0.34,2.36,17.77,X40,NTT,IT
75,TMPV,600.00,0.36,-23.82,61.42,22.98,105702.0,-53801.0,172097.0,-20.63,63.0,X-LC,5.26,3.0,-0.51,1.19,8.24,XY24,NTT,AUTO
81,VBL,671.64,-1.54,-1.79,38.14,35.67,118309.0,-5646.0,310196.0,-13.42,56.0,X-LC,2.56,4.0,-0.05,2.15,11.82,X40N,ATH,FMCG
43,ITC,452.00,0.93,-13.41,27.94,10.78,58141.0,-32224.0,208094.0,-48.51,19.0,X-LC,3.73,5.0,-0.55,1.44,0.93,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.02,-33.39,99.49,32.89,51169.0,-25776.0,51431.0,-734.87,71.0,L-MC,6.91,259.0,-0.50,0.36,41.64,XR,NTT,BANKS
49,MASFIN,398.61,-0.85,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,55.0,H-SC,6.20,164.0,-0.05,0.67,40.65,XR,ATH,FINANCE
13,BSOFT,831.70,-1.12,-22.55,94.69,50.79,103151.0,-31717.0,108936.0,-0.66,51.0,H-SC,10.80,151.0,-0.31,0.76,26.84,XR,ATH,IT
69,SURYODAY,216.00,-1.67,-19.37,52.59,23.03,75935.0,-34680.0,144391.0,57.89,53.0,H-SC,5.52,167.0,-0.46,1.00,43.26,XR,NTT,BANKS
17,COALINDIA,484.83,-0.27,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,82.0,L-LC,5.01,179.0,0.35,1.10,27.27,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.52,-29.61,60.40,12.90,92892.0,-64701.0,153795.0,127.16,68.0,H-SC,11.42,161.0,-0.70,1.07,27.88,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-0.27,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,82.0,L-LC,5.01,179.0,0.35,1.10,27.27,XY25,ATH,MINING
74,TITAGARH,1548.00,-0.40,-16.51,74.56,45.74,150753.0,-39987.0,202190.0,11.21,62.0,H-SC,3.34,174.0,-0.27,1.40,31.50,XY24,NTT,ENGINEERING
82,VOLTAS,1530.00,1.98,14.10,4.90,19.69,10720.0,27033.0,218775.0,2.93,67.0,X-MC,2.97,78.0,2.52,1.52,21.85,XY25,NTT,AC
36,ICICIPRULI,790.00,1.37,13.78,14.89,30.73,33377.0,27153.0,224158.0,-12.78,77.0,X-MC,5.44,75.0,0.81,1.55,28.31,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.02
2,50,76.41


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.88
MC    30.94
LC    26.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.77
X40      23.72
X40N     12.77
XY25     10.72
XR        8.95
AR        8.63
OX40N     7.49
SR        0.97
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.42
H-SC    22.84
X-LC    22.75
M-SC    11.34
X-SC     7.89
H-MC     4.71
M-MC     1.45
M-LC     1.27
L-LC     1.10
H-LC     1.08
L-SC     0.81
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.96,-7.69,43.03
IT,12.64,-17.40,76.62
FINANCE,11.09,-11.43,58.20
MISC,7.02,-28.51,81.07
ELECTRICAL,5.88,-10.68,51.22
PAINTS,5.68,-12.80,29.56
INSURANCE,4.74,0.76,33.20
PHARMA,3.95,-3.33,35.86
AUTO,2.86,-26.98,71.01


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3113610.0,21
AR,1329552.0,10
XR,1201503.0,12
X40,1099037.0,15
X40N,900440.0,9
OX40N,712718.0,10
XY25,348548.0,7
SR,280100.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3502776.0,24
X-MC,1464429.0,16
M-SC,1425099.0,15
X-LC,941588.0,13
X-SC,780780.0,8
H-MC,400323.0,3
L-SC,174892.0,2
M-LC,108900.0,1
H-LC,78169.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1217800.0      6
           AR         913908.0      5
M-SC       XY24       810908.0      6
H-SC       XR         779891.0      7
X-MC       X40        492078.0      7
           XY24       427009.0      3
X-LC       X40        421316.0      6
X-MC       X40N       372105.0      4
M-SC       OX40N      311463.0      5
H-SC       OX40N      311077.0      4
X-SC       X40N       305912.0      3
           XY24       289225.0      3
H-SC       SR         280100.0      2
X-LC       X40N       222423.0      2
H-MC       AR         211576.0      2
           XY24       188747.0      1
X-SC       X40        185643.0      2
X-LC       XY24       179921.0      2
M-SC       XR         176829.0      2
X-MC       XY25       173237.0      2
M-SC       AR         125899.0      2
X-LC       XY25       117928.0      3
M-LC       XR         108900.0      1
L-SC       OX40N       90178.0      1
           XR          84714.0      1
H-LC       AR          78169.0      1
L-MC       XR          51169.0      1
M-MC       XY25        35893.0      1
L-LC       XY25        21490.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
